In [1]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.external import tifffile
from skimage import transform as tf

# beneath is a list of useful functions
def crop_cms(image,threshold = 85,crop_size = 200):
    '''
    get an area around the centroid of an image
    
    image (2d numpy array): original image waiting to be cropped
    threshold (float): pixels under this threshold will be included in the centroid finding algorithm
    crop_size (int) : the half size of the image
    '''
    # get center of mass
    cms = ndimage.center_of_mass(image<threshold)
    
    # set cropped image size
    xcrop = crop_size
    ycrop = crop_size
    
    # insert a dark frame if tracking fails
    if np.isnan(cms[0]) or np.isnan(cms[1]):
        return np.zeros((xcrop*2, ycrop*2))
    
    # convert center of mass to an integer
    xcm = int(cms[0])
    ycm = int(cms[1])

    # get cropped image boundries
    xl = xcm - xcrop
    xr = xcm + xcrop
    yl = ycm - ycrop
    yr = ycm + ycrop

    # set the correct location in the cropped image
    xst = 0
    xend = xcrop * 2
    yst = 0 
    yend = ycrop * 2

    if xl<0:
        xst = - xl
        xl = 0
    if xr > image.shape[0]:
        xend -= (xr - image.shape[0])
        xr = image.shape[0]
    if yl<0:
        yst = - yl
        yl = 0
    if yr > image.shape[1]:
        yend -= (yr - image.shape[1])
        yr = image.shape[1]

    # set the cropped image
    image_crop = np.zeros((xcrop*2, ycrop*2))
    image_crop[xst:xend,yst:yend] = image[xl:xr,yl:yr]
    return image_crop

def load_avi(fname, nframes = 0, start_frame = 1):
    '''
    load an avi movie to numpy array
    
    fname (str): path to the avi file to be loaded
    nframes (int): number of frames to be loaded, if nframes is 0, will load all frames in the file
    '''
    cap = cv2.VideoCapture(fname)

    # get number of frames in the movie
    all_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # make sure number of frames is correct under default or extreme conditions
    if nframes == 0 or nframes > all_frames:
        nframes = all_frames
        
    if start_frame>= nframes:
        start_frame = 1

    #get the first frame to get the size of the frame
    ret,frame0rgb = cap.read()
    frame0 = cv2.cvtColor(frame0rgb,cv2.COLOR_BGR2GRAY)
    frame_shape = frame0.shape
    x_pix = frame_shape[0]
    y_pix = frame_shape[1]
    print('The video dimension is x = %dpx y = %dpx n = %dframes' % (x_pix,y_pix,nframes))

    #intiate numpy array
    mov = np.zeros((x_pix,y_pix,nframes-start_frame))
    mov[:,:,0] = frame0

    #load video to numpy array
    for i in range(1,nframes):
        ret, frame_rgb = cap.read()
        if i >= start_frame:
            frame = cv2.cvtColor(frame_rgb,cv2.COLOR_BGR2GRAY)
            mov[:,:,i-start_frame] = frame
        if i % 10 == 0:
            print('Loading in progress %.2f%% of the video is loaded'%(i*100.0/nframes),end = '\r')

    print('Loading is all done! %.2f%% of the video is loaded'%100)
    return mov

def transform_avi(fname, transform, out_shape, nframes = 0, start_frame = 1):
    '''
    load an avi movie to numpy array
    
    fname (str): path to the avi file to be loaded
    nframes (int): number of frames to be loaded, if nframes is 0, will load all frames in the file
    '''
    cap = cv2.VideoCapture(fname)

    # get number of frames in the movie
    all_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # make sure number of frames is correct under default or extreme conditions
    if nframes == 0 or nframes > all_frames:
        nframes = all_frames
        
    if start_frame>= nframes:
        start_frame = 1

    #get the first frame to get the size of the frame
    ret,frame0rgb = cap.read()
    frame0 = cv2.cvtColor(frame0rgb,cv2.COLOR_BGR2GRAY)
    frame_shape = frame0.shape
    x_pix = frame_shape[0]
    y_pix = frame_shape[1]
    print('The video dimension is x = %dpx y = %dpx n = %dframes' % (x_pix,y_pix,nframes))

    #intiate numpy array
    mov = np.zeros((out_shape[0],out_shape[1],nframes-start_frame))
    warped = tf.warp(frame0, transform, output_shape=out_shape)
    mov[:,:,0] = warped

    #load video to numpy array
    for i in range(1,nframes):
        ret, frame_rgb = cap.read()
        if i >= start_frame:
            frame = cv2.cvtColor(frame_rgb,cv2.COLOR_BGR2GRAY)
            warped = tf.warp(frame, transform, output_shape=out_shape)
            warped = (warped/warped.max() * 255)
            mov[:,:,i-start_frame] = warped
        if i % 10 == 0:
            print('Transforming in progress %.2f%% of the video is loaded'%(i*100.0/nframes),end = '\r')

    print('Transforming is all done! %.2f%% of the video is loaded'%100)
    return mov

In [2]:
'''
Set parameters
'''
data_folder = './data/'
sample_folder = 'track_ant_shit/'
zoomed_view_name = 'track_mov-0000.avi'


# choose crop size
csize = 200

#set the start frame
sframe = 0

#threshold for ant darkness
thrs = 85

'''
Start transforming
'''
# load raw movie
print('loading tracking view')
mov = load_avi(data_folder +sample_folder + zoomed_view_name,start_frame = sframe)
nframes = mov.shape[2]

# start to crop the movie
print('Generating zoomed view...')
mov_crop = np.zeros((csize*2,csize*2,nframes))
for i in range(nframes):
    mov_crop[:,:,i] = crop_cms(mov[:,:,i], threshold = thrs, crop_size = csize)
    if i % 60 == 0:
        print('Cropping in progress %.2f%% of the video is cropped'%(i*100.0/nframes),end = '\r')

print('Cropping is all done! %.2f%% of the video is cropped'%100)

# save the cropeed zoomed view
print('Saving zoomed view...')
mov_crop = mov_crop.transpose((2,0,1))
tifffile.imsave(data_folder +sample_folder + 'zoomed_view.tif', mov_crop.astype(np.uint8))
print('Saving complete.')

#creating transformation
print('Genrating projective transformation matrix...')
src = np.array([[0, 485], [0, 0], [485, 485],[485,0]])
dst = np.array([[391, 403], [1448, 454], [10, 1190],[1909,1173]])

tform = tf.ProjectiveTransform()
tform.estimate(src, dst)

# #load and transform wide movie
# print('Projecting side view to wide view...')
# mov_wide = transform_avi(data_folder +sample_folder + wide_view_name,tform,out_shape = (485,485), start_frame = sframe)
# print('Projection complete. Saving wide view...')
# mov_wide = mov_wide.transpose((2,0,1))
# tifffile.imsave(data_folder +sample_folder + 'wide_view.tif', mov_wide.astype(np.uint8))
# print('Saving complete.')
# print('Process done.')

loading tracking view
The video dimension is x = 1024px y = 1024px n = 1673frames
Loading is all done! 100.00% of the video is loaded
Generating zoomed view...


C:\ProgramData\Anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1301: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


Cropping is all done! 100.00% of the video is cropped
Saving zoomed view...
Saving complete.
Genrating projective transformation matrix...


True

In [4]:
nframes

1